In [1]:
# Set GPUs
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [22]:
# Data Loading

# Import packages
import pandas as pd

# Import data
df = pd.read_csv("../Data/subject-info-cleaned-with-prognosis-sampled-B.csv")

# Get prognosis and outcome
df = df[['Prognosis', 'Outcome']]

# Map labels to integers
# Survivor = 0, SCD = 1, PFD = 2
label_map = {"survivor": 0, "sudden cardiac death": 1, "pump failure death": 2}
df['Outcome'] = df['Outcome'].map(label_map)
df.rename(columns = {"Outcome": "labels"}, inplace = True)
df.head()

# Shuffle dataset
# df = df.sample(frac = 1, random_state = 42)
# df.head()

,Prognosis,labels
0,REASONING: \n\nThe patient's outcome of pump f...,2
1,REASONING: \n\nThe patient's outcome of pump f...,2
2,REASONING: \n\nThe patient outcome of pump fai...,2
3,REASONING: The patient's outcome of pump failu...,2
4,REASONING: The patient's outcome of pump failu...,2


In [23]:
# Load in BioBERT with classification head
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch import cuda

model_name = "dmis-lab/biobert-base-cased-v1.1"
num_labels = 3  # Three possible outcomes

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

device = 'cuda' if cuda.is_available() else 'cpu'
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [26]:
# Data split (for testing)
from sklearn.model_selection import train_test_split

# Split data
train_df, test_df = train_test_split(df, test_size = 0.3, random_state = 42, stratify = df['labels'])

# Get encodings
train_encodings = tokenizer(list(train_df['Prognosis']), truncation = True, padding = True)
test_encodings = tokenizer(list(test_df['Prognosis']), truncation = True, padding = True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [27]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(df)


In [28]:
# Load BioBERT tokenizer
model_name = "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Prognosis"], padding="max_length", truncation=True, max_length = 512)

# Apply tokenization
dataset = dataset.map(tokenize_function, batched=True)


Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 1508.83 examples/s]


In [29]:
# Split data
train_test = dataset.train_test_split(test_size=0.2)
train_dataset = train_test["train"]
val_dataset = train_test["test"]


In [35]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Convert logits to class predictions

    # Compute accuracy
    acc = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")

    # Compute AUC (only if there are at least 2 classes)
    # auc = roc_auc_score(labels, logits, multi_class="ovr") if len(set(labels)) > 1 else 0

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
        #"auc": auc
    }


In [30]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="../Results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="../logs",
    logging_steps=10,
    load_best_model_at_end=True,
)


/home/sswee/myenv/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [36]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

trainer.train()


/tmp/ipykernel_3496970/360158454.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/sswee/myenv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.896306,0.444444,0.462963,0.444444,0.412698
2,No log,0.760180,0.777778,0.925926,0.777778,0.817460
3,No log,0.699405,0.777778,0.925926,0.777778,0.817460


/home/sswee/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sswee/myenv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/sswee/myenv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/sswee/myenv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vecto

TrainOutput(global_step=9, training_loss=0.6715312004089355, metrics={'train_runtime': 25.7372, 'train_samples_per_second': 4.196, 'train_steps_per_second': 0.35, 'total_flos': 28416249114624.0, 'train_loss': 0.6715312004089355, 'epoch': 3.0})

In [37]:
trainer.evaluate()

/home/sswee/myenv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.6994049549102783,
 'eval_accuracy': 0.7777777777777778,
 'eval_precision': 0.925925925925926,
 'eval_recall': 0.7777777777777778,
 'eval_f1': 0.8174603174603174,
 'eval_runtime': 0.0729,
 'eval_samples_per_second': 123.451,
 'eval_steps_per_second': 13.717,
 'epoch': 3.0}